In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def load_images(dirname, filetype=None, grayscale=False, size=None):
    mode = 0 if grayscale else 1
    images = []
    filenames = os.listdir(dirname)
    
    if filetype is not None:
        filenames = [filename for filename in filenames if filename.endswith('.' + filetype)]
        
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename), mode)
        
        if size is not None:
            image = cv2.resize(image, size)
        
        images.append(image)
    
    return images

In [3]:
def prepare_data(c, nc, expand_dims=False, normalization_method=None, return_y=False, y_categorical=False):
    c  = np.array(c) if type(c) == list else c
    nc = np.array(nc) if type(nc) == list else nc
    
    X = np.concatenate((c, nc), axis=0)
    
    if expand_dims:
        X = np.expand_dims(X, axis=3)
    
    if normalization_method is not None:
        X = X.astype(np.float32)
        
        if normalization_method == 1:
            X = (X - X.min()) / (X.max() - X.min())
        
        elif normalization_method == 2:
            X = 2 * (X - X.min()) / (X.max() - X.min())
            
        elif normalization_method == 3:
            X = (X - X.mean()) / X.std()
    
    if return_y:
        c_label  = np.ones(c.shape[0])
        nc_label = np.zeros(nc.shape[0])
        
        y = np.concatenate((c_label, nc_label), axis=0)
        
        if y_categorical:
            y = to_categorical(y)
        
        return X, y
    
    return X

In [4]:
def evaluate_multiple_weights(model_builder, times, weights_path_format, **kwargs):
    results = []
    
    for t in range(times):
        weights_path = weights_path_format.format(t + 1)
        
        print('Avaliando o modelo com os pesos {}'.format(weights_path))
        
        model = model_builder()
        
        model.load_weights(weights_path)
        
        result = model.evaluate(**kwargs)
        
        results.append(result)

    return results

In [5]:
def show_results_df(results, describe=False):
    results_df = pd.DataFrame(results, columns=['Loss', 'Accuracy'])
    
    results_df.index += 1
    
    if describe:
        return results_df.describe()
    
    else:
        return results_df

## Modelos

### Modelo 1

In [6]:
def model_1():
    net_input = Input(shape=(128, 128, 1))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 2

In [7]:
def model_2():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 3

In [8]:
def model_3():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_output = Dense(2, activation='softmax')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 4

In [9]:
def model_4():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = BatchNormalization()(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 5

In [10]:
def model_5():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 6

In [11]:
def model_6():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(128, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 7

In [12]:
def model_7():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(256, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 8

In [13]:
def model_8():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(64, (3, 3), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(256, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 9

In [14]:
def model_9():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(64, (5, 5), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(256, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 10

In [15]:
def model_10():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(64, (3, 3), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (3, 3), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(256, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

### Modelo 11

In [16]:
def model_11():
    net_input = Input(shape=(128, 128, 3))
    
    net_layer = Conv2D(32, (3, 3), activation='relu')(net_input)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(64, (5, 5), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (7, 7), activation='relu')(net_layer)
    net_layer = MaxPooling2D((2, 2))(net_layer)
    net_layer = BatchNormalization()(net_layer)
    net_layer = Dropout(0.25)(net_layer)
    
    net_layer = Conv2D(128, (1,1))(net_layer)
    net_layer = GlobalMaxPooling2D()(net_layer)
    
    net_layer = Dense(256, activation='relu')(net_layer)
    net_layer = Dropout(0.5)(net_layer)
    
    net_output = Dense(1, activation='sigmoid')(net_layer)
    
    model = Model(inputs=net_input, outputs=net_output)
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Avaliação

### Arquitetura 1

In [17]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', grayscale=True, size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', grayscale=True, size=(128, 128))

In [18]:
X_test, y_test = prepare_data(c_test, nc_test, expand_dims=True, normalization_method=1, return_y=True)

In [19]:
architecture_1_results = evaluate_multiple_weights(model_builder=model_1, times=10,
                                                   weights_path_format='weights/architecture_1/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_1/training_1.hdf5
1336/1336 [==============================] - 14s 11ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_2.hdf5
1336/1336 [==============================] - 11s 8ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_3.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_4.hdf5
1336/1336 [==============================] - 16s 12ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_5.hdf5
1336/1336 [==============================] - 11s 8ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_6.hdf5
1336/1336 [==============================] - 11s 8ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_7.hdf5
1336/1336 [==============================] - 11s 9ms/step
Avaliando o modelo com os pesos weights/architecture_1/training_8.hdf5
1336/1336 [============

In [20]:
show_results_df(architecture_1_results)

,Loss,Accuracy
1,1.063956,0.5
2,0.992195,0.5
3,1.021511,0.5
4,0.752080,0.5
5,1.099998,0.5
6,0.969572,0.5
7,0.945639,0.5
8,0.936306,0.5
9,0.872787,0.5
10,0.842139,0.5


In [21]:
show_results_df(architecture_1_results, describe=True)

,Loss,Accuracy
count,10.000000,10.0
mean,0.949618,0.5
std,0.105270,0.0
min,0.752080,0.5
25%,0.888667,0.5
50%,0.957605,0.5
75%,1.014182,0.5
max,1.099998,0.5


### Arquitetura 2

In [22]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [23]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=1, return_y=True)

In [24]:
architecture_2_results = evaluate_multiple_weights(model_builder=model_2, times=10,
                                                   weights_path_format='weights/architecture_2/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_2/training_1.hdf5
1336/1336 [==============================] - 8s 6ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_2.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_3.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_4.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_5.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_6.hdf5
1336/1336 [==============================] - 7s 6ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_7.hdf5
1336/1336 [==============================] - 7s 6ms/step
Avaliando o modelo com os pesos weights/architecture_2/training_8.hdf5
1336/1336 [=====================

In [25]:
show_results_df(architecture_2_results)

,Loss,Accuracy
1,1.027502,0.775449
2,0.723167,0.500000
3,0.284006,0.890719
4,0.507106,0.723054
5,0.519164,0.857784
6,0.484520,0.841317
7,1.226472,0.500000
8,0.610360,0.821856
9,1.123815,0.500000
10,0.466113,0.801647


In [26]:
show_results_df(architecture_2_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.697223,0.721183
std,0.319138,0.159220
min,0.284006,0.500000
25%,0.490167,0.555763
50%,0.564762,0.788548
75%,0.951419,0.836452
max,1.226472,0.890719


### Arquitetura 3

In [27]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [28]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=1, return_y=True, y_categorical=True)

In [29]:
architecture_3_results = evaluate_multiple_weights(model_builder=model_3, times=10,
                                                   weights_path_format='weights/architecture_3/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_3/training_1.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_2.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_3.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_4.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_5.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_6.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_7.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_3/training_8.hdf5
1336/1336 [======================

In [30]:
show_results_df(architecture_3_results)

,Loss,Accuracy
1,0.793165,0.500000
2,0.300115,0.872006
3,0.641540,0.517964
4,2.000733,0.500000
5,0.425842,0.878743
6,1.313418,0.500000
7,1.546329,0.500000
8,0.400564,0.827096
9,0.791236,0.500000
10,0.620607,0.529940


In [31]:
show_results_df(architecture_3_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.883355,0.612575
std,0.557746,0.171047
min,0.300115,0.500000
25%,0.474534,0.500000
50%,0.716388,0.508982
75%,1.183355,0.752807
max,2.000733,0.878743


### Arquitetura 4

In [32]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [33]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=2, return_y=True)

In [34]:
architecture_4_results = evaluate_multiple_weights(model_builder=model_2, times=10,
                                                   weights_path_format='weights/architecture_4/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_4/training_1.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_2.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_3.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_4.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_5.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_6.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_7.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_4/training_8.hdf5
1336/1336 [======================

In [35]:
show_results_df(architecture_4_results)

,Loss,Accuracy
1,0.491432,0.866766
2,0.382551,0.848054
3,0.598055,0.828593
4,0.419838,0.824102
5,0.624843,0.657934
6,0.677937,0.512725
7,0.347476,0.843563
8,0.786716,0.500000
9,0.597257,0.843563
10,0.810857,0.500000


In [36]:
show_results_df(architecture_4_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.573696,0.722530
std,0.161385,0.161471
min,0.347476,0.500000
25%,0.437736,0.549027
50%,0.597656,0.826347
75%,0.664664,0.843563
max,0.810857,0.866766


### Arquitetura 5

In [37]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [38]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [39]:
architecture_5_results = evaluate_multiple_weights(model_builder=model_2, times=10,
                                                   weights_path_format='weights/architecture_5/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_5/training_1.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_2.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_3.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_4.hdf5
1336/1336 [==============================] - 7s 6ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_5.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_6.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_7.hdf5
1336/1336 [==============================] - 7s 5ms/step
Avaliando o modelo com os pesos weights/architecture_5/training_8.hdf5
1336/1336 [======================

In [40]:
show_results_df(architecture_5_results)

,Loss,Accuracy
1,0.455302,0.732784
2,0.450953,0.755240
3,0.675900,0.558383
4,0.419412,0.767964
5,0.427464,0.769461
6,0.422836,0.789671
7,0.349385,0.876497
8,0.360126,0.833084
9,0.563222,0.698353
10,0.385982,0.841317


In [41]:
show_results_df(architecture_5_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.451058,0.762275
std,0.099078,0.089269
min,0.349385,0.558383
25%,0.394340,0.738398
50%,0.425150,0.768713
75%,0.454215,0.822231
max,0.675900,0.876497


### Arquitetura 6

In [42]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [43]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [44]:
architecture_6_results = evaluate_multiple_weights(model_builder=model_4, times=10,
                                                   weights_path_format='weights/architecture_6/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_6/training_1.hdf5
1336/1336 [==============================] - 48s 36ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_2.hdf5
1336/1336 [==============================] - 58s 44ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_3.hdf5
1336/1336 [==============================] - 51s 38ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_4.hdf5
1336/1336 [==============================] - 46s 35ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_5.hdf5
1336/1336 [==============================] - 45s 34ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_6.hdf5
1336/1336 [==============================] - 45s 33ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_7.hdf5
1336/1336 [==============================] - 45s 33ms/step
Avaliando o modelo com os pesos weights/architecture_6/training_8.hdf5
1336/1336 [========

In [45]:
show_results_df(architecture_6_results)

,Loss,Accuracy
1,0.233001,0.916916
2,0.207088,0.934132
3,0.223329,0.919910
4,0.200491,0.917665
5,0.230668,0.910928
6,0.206413,0.935629
7,0.188443,0.935629
8,0.242152,0.907186
9,0.213104,0.918413
10,0.252023,0.912425


In [46]:
show_results_df(architecture_6_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.219671,0.920883
std,0.019956,0.010550
min,0.188443,0.907186
25%,0.206582,0.913548
50%,0.218216,0.918039
75%,0.232418,0.930576
max,0.252023,0.935629


### Arquitetura 7

In [47]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [48]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [49]:
architecture_7_results = evaluate_multiple_weights(model_builder=model_5, times=10,
                                                   weights_path_format='weights/architecture_7/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_7/training_1.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_2.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_3.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_4.hdf5
1336/1336 [==============================] - 10s 8ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_5.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_6.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_7.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_7/training_8.hdf5
1336/1336 [===================

In [50]:
show_results_df(architecture_7_results)

,Loss,Accuracy
1,0.207412,0.932635
2,0.219422,0.921407
3,0.220278,0.926647
4,0.208210,0.937874
5,0.197964,0.934880
6,0.225024,0.916168
7,0.276298,0.904192
8,0.191749,0.939371
9,0.177059,0.944611
10,0.222114,0.923653


In [51]:
show_results_df(architecture_7_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.214553,0.928144
std,0.026516,0.012213
min,0.177059,0.904192
25%,0.200326,0.921969
50%,0.213816,0.929641
75%,0.221655,0.937126
max,0.276298,0.944611


### Arquitetura 8

In [52]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [53]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [54]:
architecture_8_results = evaluate_multiple_weights(model_builder=model_6, times=10,
                                                   weights_path_format='weights/architecture_8/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_8/training_1.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_2.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_3.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_4.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_5.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_6.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_7.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_8/training_8.hdf5
1336/1336 [=====================

In [55]:
show_results_df(architecture_8_results)

,Loss,Accuracy
1,0.133848,0.948353
2,0.129573,0.949102
3,0.129216,0.947605
4,0.133364,0.950599
5,0.150312,0.949102
6,0.161184,0.936377
7,0.139732,0.939371
8,0.142376,0.949850
9,0.151680,0.943114
10,0.140834,0.947605


In [56]:
show_results_df(architecture_8_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.141212,0.946108
std,0.010489,0.004838
min,0.129216,0.936377
25%,0.133485,0.944237
50%,0.140283,0.947979
75%,0.148328,0.949102
max,0.161184,0.950599


### Arquitetura 9

In [57]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [58]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [59]:
architecture_9_results = evaluate_multiple_weights(model_builder=model_7, times=10,
                                                   weights_path_format='weights/architecture_9/training_{}.hdf5',
                                                   x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_9/training_1.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_2.hdf5
1336/1336 [==============================] - 9s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_3.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_4.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_5.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_6.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_7.hdf5
1336/1336 [==============================] - 10s 7ms/step
Avaliando o modelo com os pesos weights/architecture_9/training_8.hdf5
1336/1336 [=================

In [60]:
show_results_df(architecture_9_results)

,Loss,Accuracy
1,0.110735,0.954341
2,0.119788,0.952096
3,0.139065,0.949850
4,0.129676,0.963323
5,0.144998,0.944611
6,0.120049,0.953593
7,0.121365,0.955090
8,0.142581,0.945359
9,0.139914,0.945359
10,0.141431,0.953593


In [61]:
show_results_df(architecture_9_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.130960,0.951722
std,0.012176,0.005725
min,0.110735,0.944611
25%,0.120378,0.946482
50%,0.134371,0.952844
75%,0.141052,0.954154
max,0.144998,0.963323


### Arquitetura 10

In [62]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [63]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [64]:
architecture_10_results = evaluate_multiple_weights(model_builder=model_8, times=10,
                                                    weights_path_format='weights/architecture_10/training_{}.hdf5',
                                                    x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_10/training_1.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_2.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_3.hdf5
1336/1336 [==============================] - 14s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_4.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_5.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_6.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_7.hdf5
1336/1336 [==============================] - 15s 11ms/step
Avaliando o modelo com os pesos weights/architecture_10/training_8.hdf5
1336/1336 [

In [65]:
show_results_df(architecture_10_results)

,Loss,Accuracy
1,0.097985,0.962575
2,0.104935,0.964072
3,0.097371,0.965569
4,0.108968,0.961826
5,0.126814,0.954341
6,0.094585,0.967814
7,0.097341,0.970808
8,0.108867,0.956587
9,0.094359,0.971557
10,0.089962,0.970808


In [66]:
show_results_df(architecture_10_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.102119,0.964596
std,0.010737,0.005947
min,0.089962,0.954341
25%,0.095274,0.962013
50%,0.097678,0.964820
75%,0.107884,0.970060
max,0.126814,0.971557


### Arquitetura 11

In [67]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [68]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [69]:
architecture_11_results = evaluate_multiple_weights(model_builder=model_9, times=10,
                                                    weights_path_format='weights/architecture_11/training_{}.hdf5',
                                                    x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_11/training_1.hdf5
1336/1336 [==============================] - 26s 19ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_2.hdf5
1336/1336 [==============================] - 24s 18ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_3.hdf5
1336/1336 [==============================] - 23s 17ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_4.hdf5
1336/1336 [==============================] - 30s 22ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_5.hdf5
1336/1336 [==============================] - 28s 21ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_6.hdf5
1336/1336 [==============================] - 32s 24ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_7.hdf5
1336/1336 [==============================] - 28s 21ms/step
Avaliando o modelo com os pesos weights/architecture_11/training_8.hdf5
1336/1336 [

In [70]:
show_results_df(architecture_11_results)

,Loss,Accuracy
1,0.077166,0.973054
2,0.080601,0.975299
3,0.099345,0.968563
4,0.087417,0.970060
5,0.084364,0.972305
6,0.076167,0.975299
7,0.076012,0.976048
8,0.098763,0.967066
9,0.082456,0.970060
10,0.083268,0.974551


In [71]:
show_results_df(architecture_11_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.084556,0.972231
std,0.008488,0.003145
min,0.076012,0.967066
25%,0.078025,0.970060
50%,0.082862,0.972680
75%,0.086654,0.975112
max,0.099345,0.976048


### Arquitetura 12

In [72]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [73]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [74]:
architecture_12_results = evaluate_multiple_weights(model_builder=model_10, times=10,
                                                    weights_path_format='weights/architecture_12/training_{}.hdf5',
                                                    x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_12/training_1.hdf5
1336/1336 [==============================] - 21s 16ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_2.hdf5
1336/1336 [==============================] - 21s 16ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_3.hdf5
1336/1336 [==============================] - 22s 16ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_4.hdf5
1336/1336 [==============================] - 29s 22ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_5.hdf5
1336/1336 [==============================] - 24s 18ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_6.hdf5
1336/1336 [==============================] - 29s 22ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_7.hdf5
1336/1336 [==============================] - 23s 17ms/step
Avaliando o modelo com os pesos weights/architecture_12/training_8.hdf5
1336/1336 [

In [75]:
show_results_df(architecture_12_results)

,Loss,Accuracy
1,0.075829,0.974551
2,0.073412,0.974551
3,0.078944,0.973054
4,0.088841,0.970808
5,0.071411,0.973802
6,0.069408,0.976796
7,0.073838,0.976048
8,0.072379,0.977545
9,0.070523,0.976048
10,0.064751,0.977545


In [76]:
show_results_df(architecture_12_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.073934,0.975075
std,0.006467,0.002148
min,0.064751,0.970808
25%,0.070745,0.973990
50%,0.072896,0.975299
75%,0.075331,0.976609
max,0.088841,0.977545


### Arquitetura 13

In [77]:
c_test  = load_images('Dataset_2/Test/C', filetype='tif', size=(128, 128))
nc_test = load_images('Dataset_2/Test/NC', filetype='tif', size=(128, 128))

In [78]:
X_test, y_test = prepare_data(c_test, nc_test, normalization_method=3, return_y=True)

In [79]:
architecture_13_results = evaluate_multiple_weights(model_builder=model_11, times=10,
                                                    weights_path_format='weights/architecture_13/training_{}.hdf5',
                                                    x=X_test, y=y_test)

Avaliando o modelo com os pesos weights/architecture_13/training_1.hdf5
1336/1336 [==============================] - 57s 43ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_2.hdf5
1336/1336 [==============================] - 55s 41ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_3.hdf5
1336/1336 [==============================] - 54s 40ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_4.hdf5
1336/1336 [==============================] - 53s 39ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_5.hdf5
1336/1336 [==============================] - 58s 43ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_6.hdf5
1336/1336 [==============================] - 58s 43ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_7.hdf5
1336/1336 [==============================] - 54s 40ms/step
Avaliando o modelo com os pesos weights/architecture_13/training_8.hdf5
1336/1336 [

In [80]:
show_results_df(architecture_13_results)

,Loss,Accuracy
1,0.061392,0.979790
2,0.068841,0.980539
3,0.070808,0.981287
4,0.058939,0.985030
5,0.059970,0.984281
6,0.080555,0.982036
7,0.063001,0.984281
8,0.072188,0.982784
9,0.072119,0.976048
10,0.056786,0.983533


In [81]:
show_results_df(architecture_13_results, describe=True)

,Loss,Accuracy
count,10.000000,10.000000
mean,0.066460,0.981961
std,0.007587,0.002698
min,0.056786,0.976048
25%,0.060325,0.980726
50%,0.065921,0.982410
75%,0.071791,0.984094
max,0.080555,0.985030
